In [0]:
#dbfs:/FileStore/shared_uploads/danghia2002@gmail.com/winequality_red.csv

In [0]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml import Pipeline

In [0]:
#Tạo Spark session
spark = SparkSession.builder.appName("WineQualityPrediction").getOrCreate()


In [0]:

#Đọc dữ liệu từ CSV
data = spark.read.csv("dbfs:/FileStore/shared_uploads/danghia2002@gmail.com/winequality_red.csv", header=True, inferSchema=True)


In [0]:
#Tạo vector assembler để chuyển các đặc trưng thành một cột vector
feature_columns = ["fixed acidity", "volatile acidity", "citric acid", "residual sugar", "chlorides",
                   "free sulfur dioxide", "total sulfur dioxide", "density", "pH", "sulphates", "alcohol"]

assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
data = assembler.transform(data)


In [0]:

#Chia dữ liệu thành tập huấn luyện và tập kiểm tra (tỉ lệ 70/30)
(train_data, test_data) = data.randomSplit([0.7, 0.3], seed=42)

In [0]:
#Tạo mô hình Decision Tree
dt = DecisionTreeClassifier(labelCol="quality", featuresCol="features")

In [0]:

#Tạo pipeline
pipeline = Pipeline(stages=[dt])



In [0]:

#Huấn luyện mô hình trên tập huấn luyện
model = pipeline.fit(train_data)


In [0]:

#Dự đoán trên tập kiểm tra
predictions = model.transform(test_data)


In [0]:

#Đánh giá độ chính xác của mô hình
evaluator = MulticlassClassificationEvaluator(labelCol="quality", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print(f"Accuracy: {accuracy}")


Accuracy: 0.568075117370892


In [0]:

#Thực hiện điều chỉnh tham số của mô hình
param_grid = ParamGridBuilder() \
    .addGrid(dt.maxDepth, [5, 10, 15]) \
    .addGrid(dt.maxBins, [20, 40, 60]) \
    .build()

crossval = CrossValidator(estimator=dt, estimatorParamMaps=param_grid, evaluator=evaluator, numFolds=3)
cv_model = crossval.fit(train_data)


In [0]:

#Dự đoán trên tập kiểm tra
cv_predictions = cv_model.transform(test_data)


In [0]:

#Đánh giá độ chính xác của mô hình sau điều chỉnh tham số
cv_accuracy = evaluator.evaluate(cv_predictions)
print(f"Accuracy after tuning: {cv_accuracy}")
best_model = cv_model.bestModel
print(f"Best Max Depth: {best_model.getMaxDepth()}")
print(f"Best Max Bins: {best_model.getMaxBins()}")

Accuracy after tuning: 0.5727699530516432
Best Max Depth: 5
Best Max Bins: 40


In [0]:
from pyspark.ml.classification import RandomForestClassifier


In [0]:
#Tạo mô hình Random Forest
rf = RandomForestClassifier(labelCol="quality", featuresCol="features")

In [0]:

#Tạo pipeline
pipeline = Pipeline(stages=[rf])

In [0]:
#Huấn luyện mô hình trên tập huấn luyện
model = pipeline.fit(train_data)



In [0]:

#Dự đoán trên tập kiểm tra
predictions = model.transform(test_data)


In [0]:

#Đánh giá độ chính xác của mô hình
evaluator = MulticlassClassificationEvaluator(labelCol="quality", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print(f"Accuracy: {accuracy}")


Accuracy: 0.6103286384976526


In [0]:

#Thực hiện điều chỉnh tham số của mô hình
param_grid = ParamGridBuilder() \
    .addGrid(rf.numTrees, [10, 20, 30]) \
    .addGrid(rf.maxDepth, [5, 10, 15]) \
    .build()
crossval = CrossValidator(estimator=rf, estimatorParamMaps=param_grid, evaluator=evaluator, numFolds=3)
cv_model = crossval.fit(train_data)


In [0]:


#Dự đoán trên tập kiểm tra
cv_predictions = cv_model.transform(test_data)

In [0]:

#Đánh giá độ chính xác của mô hình sau điều chỉnh tham số
cv_accuracy = evaluator.evaluate(cv_predictions)
print(f"Accuracy after tuning: {cv_accuracy}")
best_model = cv_model.bestModel
print(f"Best Num Trees: {best_model.getNumTrees}")
print(f"Best Max Depth: {best_model.getOrDefault('maxDepth')}")

Accuracy after tuning: 0.6643192488262911
Best Num Trees: 30
Best Max Depth: 10
